# Useful Aggregations on Timeseries Data with Pandas

While writing this  blog article, I took a break from working on lots of time series data with Pandas. I was performing lots of aggregations and feature engineering tasks on top of a Credit Card Transaction dataset. I want to share with you some of my insights in usefull operations for performing explorative data analysis or preparing a times series dataset to perform some machine learning task on top of it. 

In this blog post you will learn:

* How to load time series data from a csv 
* What the rolling operation on a dataframe is what is it usefull for
* How to combine group by operation and rolling operation on a pandas dataframe
* Some examples for transformations using the two operations above that will be usefull for you in practice
* Some Hints in how to parellize these operations to be using all you CPU Cores.

(Hint you can find a Jupyter notebook containing all the code and the toy data mentioned inthis blog post here: TODO)

Let us start with loading the data.

## Loading time series data

Loading timeseries data from a CSV is straight forward in pandas. We simply use the read csv comand and define the `Datetime` column as an index column and also give pandas the hint that it should parse the `Datetime` column as pandas Datetime field.

In [ ]:
import pandas as pd

In [102]:
df = pd.read_csv("time_series_example.csv",index_col="Datetime",parse_dates=["Datetime"])
df = df.sort_index()

In [103]:
result_df = df.copy()

In [104]:
df

,Card ID,Amount
Datetime,,
2019-12-01,1,72.70
2019-12-02,2,186.78
2019-12-05,1,29.20
2019-12-08,2,131.10
2019-12-12,1,30.30
2019-12-17,2,145.20
2019-12-18,1,43.70
2019-12-23,2,200.10
2019-12-26,1,189.90


We can now see that we loaded sucessfully the data. Let's take a brief look at it. For all TimeSeries operations it is critical that pandas loaded the index correctly as an `DatetimeIndex` you can validate this by typing `df.index` and see the correct index. Next two the `Datetime` index column, that refers to the timestamp of a credit card purchase(transaction), we have a `Card ID` column refering to an ID of a Credit Card and an `Amount` column, that ..., well indicates the amount in Dollar of the purchase with the card at the specified time

In [105]:
df.index

DatetimeIndex(['2019-12-01', '2019-12-02', '2019-12-05', '2019-12-08',
               '2019-12-12', '2019-12-17', '2019-12-18', '2019-12-23',
               '2019-12-26', '2019-12-27'],
              dtype='datetime64[ns]', name='Datetime', freq=None)

## Rolling Window on Timeseries with Pandas

The first thing we're interested in is what is 7 day rolling mean of the credit card transaction amounts. What this means in this example is quite easy to explain. For every single transaction we look 7 days back, collect all transactions that fall in this range and get the average of the `Amount` column. Luckily this is very easy to achieve with pandas:

In [71]:
df.rolling('7D').Amount.mean()

Datetime
2019-12-01     72.700000
2019-12-02    129.740000
2019-12-05     96.226667
2019-12-08    115.693333
2019-12-12     80.700000
2019-12-17     87.750000
2019-12-18     73.066667
2019-12-23    129.666667
2019-12-26    195.000000
2019-12-27    319.066667
Name: Amount, dtype: float64

This information might by quite interesting in some use cases, for credit card transaction use cases we usually are interested in the average revenue, the amount of transaction, etc... per customer in some time window.

## Combining grouping and rolling window timeseries aggregations with pandas

We can achieve this by grouping our dataframe by the columns `Card ID` and then perfom the rolling operation on every group individually. Here is how we get the amount of transactions in the last 7 days for any transaction for every Card seperately. (Hint we store the result in a dataframe to later merge it back to the orignal df to get on comprehensive dataframe with all the relevant data)

In [111]:
df.groupby("Card ID").rolling('7D').Amount.count()

Card ID  Datetime  
1        2019-12-01    1.0
         2019-12-05    2.0
         2019-12-12    1.0
         2019-12-18    2.0
         2019-12-26    1.0
2        2019-12-02    1.0
         2019-12-08    2.0
         2019-12-17    1.0
         2019-12-23    2.0
         2019-12-27    2.0
Name: Amount, dtype: float64

In [112]:
df_7d_count = pd.DataFrame(df.groupby("Card ID").rolling('7D').Amount.count())
df_7d_count = df_7d_count.rename(columns={"Amount":"Transaction Count 7D"})
df_7d_count

Transaction Count 7D
Card ID Datetime                        
1       2019-12-01                   1.0
        2019-12-05                   2.0
        2019-12-12                   1.0
        2019-12-18                   2.0
        2019-12-26                   1.0
2       2019-12-02                   1.0
        2019-12-08                   2.0
        2019-12-17                   1.0
        2019-12-23                   2.0
        2019-12-27                   2.0

And we might also be interested in a average transaction volume.

In [73]:
df_7d_mean_amount = pd.DataFrame(df.groupby("Card ID").rolling('7D').Amount.mean())
df_7d_mean_amount = df_7d_mean_amount.rename(columns={"Amount":"Mean Amount 7D"})
df_7d_mean_amount

## Merging the result in one comprehensive DataFrame
In order to have an overview of what features we have, we can merge now simply the two created dataframe back to an copy of the orignal dataframe.

In [99]:
result_df = df.copy()
result_df = result_df.merge(df_7d_mean_amount,left_index=True, right_index=True)
result_df = result_df.merge(df_7d_count,left_index=True, right_index=True)
result_df

Card ID  Amount  Mean Amount 7D  Transaction Count 7D
Card ID Datetime                                                         
1       2019-12-01        1   72.70           72.70                   1.0
        2019-12-05        1   29.20           50.95                   2.0
        2019-12-12        1   30.30           30.30                   1.0
        2019-12-18        1   43.70           37.00                   2.0
        2019-12-26        1  189.90          189.90                   1.0
2       2019-12-02        2  186.78          186.78                   1.0
        2019-12-08        2  131.10          158.94                   2.0
        2019-12-17        2  145.20          145.20                   1.0
        2019-12-23        2  200.10          172.65                   2.0
        2019-12-27        2  567.20          383.65                   2.0

This looks already quite good let us just add one more feature to get the average amount of transactions in 7 days by card. Therefore we have now simply group by the `Card ID` again and then get the average of the `Transaction Count 7D`.

In [95]:
df_7d_mean_count = pd.DataFrame(result_df["Transaction Count 7D"].groupby("Card ID").mean())
df_7d_mean_count = df_7d_mean_count.rename(columns={"Transaction Count 7D":"Mean 7D Transaction Count","Card ID":"Card"})
df_7d_mean_count

,Mean 7D Transaction Count
Card ID,
1,1.4
2,1.6


Since we have here now the need of joining two datasets with different indices we use the inner join then pandas picks automatically which index level we want to use and we see that now have the new column `Mean 7D Transcation Count`. We could add like this many different features to the dataset, e.g. like the maximum 7 Days Rolling Amount, minimum, etc..

In [100]:
result_df = result_df.join(df_7d_mean_count, how='inner')
result_df

Card ID  Amount  Mean Amount 7D  Transaction Count 7D  \
Card ID Datetime                                                            
1       2019-12-01        1   72.70           72.70                   1.0   
        2019-12-05        1   29.20           50.95                   2.0   
        2019-12-12        1   30.30           30.30                   1.0   
        2019-12-18        1   43.70           37.00                   2.0   
        2019-12-26        1  189.90          189.90                   1.0   
2       2019-12-02        2  186.78          186.78                   1.0   
        2019-12-08        2  131.10          158.94                   2.0   
        2019-12-17        2  145.20          145.20                   1.0   
        2019-12-23        2  200.10          172.65                   2.0   
        2019-12-27        2  567.20          383.65                   2.0   

                    Mean 7D Transaction Count  
Card ID Datetime                               
1       2019-12-01                        1.4  
        2019-12-05                        1.4  
        2019-12-12                        1.4  
        2019-12-18                        1.4  
        2019-12-26                        1.4  
2       2019-12-02                        1.6  
        2019-12-08                        1.6  
        2019-12-17                        1.6  
        2019-12-23                        1.6  
        2019-12-27                        1.6

What i find very useful we can now compute differences from the current 7 day window to the mean of all windows which can be for credit cards useful to find fraudulent transactions. 

## Parallelize Group By Rolling Aggregation Operations

For datasets with lots of different cards (or any other grouping criteria) and lots of transactions (or any other timeseries events) these operations can become very computational inefficient. The first obvious choice to is to scale up the operations on your local machine e.g. to use all the CPU Cores available in contrast to the pandas default to only use one CPU core. I find the little library pandarellel : https://github.com/nalepae/pandarallel Very usefull. I recently fixed a bug there that now it also works on time series grouped by and rolling dataframes.  Here is a small example of how to use the library to parallelize one operation

In [113]:
!pip3 install git+https://github.com/dice89/pandarallel.git#egg=pandarallel --upgrade

  Cloning https://github.com/dice89/pandarallel.git to /private/var/folders/24/1t8_x_9d3n5b8zn0p4gwfb_w0000gn/T/pip-install-hlqcm0me/pandarallel
  Running command git clone -q https://github.com/dice89/pandarallel.git /private/var/folders/24/1t8_x_9d3n5b8zn0p4gwfb_w0000gn/T/pip-install-hlqcm0me/pandarallel
     |████████████████████████████████| 153kB 102kB/s eta 0:00:01
  Created wheel for pandarallel: filename=pandarallel-1.4.2-cp36-none-any.whl size=17060 sha256=03e8e8ccba8a7a627c8688ef611abb2b26d78682dcad34d0c4982d3a225f297b
  Stored in directory: /private/var/folders/24/1t8_x_9d3n5b8zn0p4gwfb_w0000gn/T/pip-ephem-wheel-cache-bub2k95c/wheels/04/49/7f/7d4feb4233df5499edccb6288874f4aba9bda650bdb0ddd378
  Created wheel for dill: filename=dill-0.3.1.1-cp36-none-any.whl size=78532 sha256=6e214476cfd1ecb2fba9a9fcc9eed8f8e5f5f7b01f825a487d2694c57815ef16
  Stored in directory: /Users/alexandermuller/Library/Caches/pip/wheels/59/b1/91/f02e76c732915c4015ab4010f3015469866c1eb9b14058d8e7
Succes

In [116]:
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=2)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data tranfer (pipe) to transfer data between the main process and workers.


In [122]:
import numpy as np

In [128]:
df.groupby("Card ID").rolling('7D').Amount.parallel_apply(np.mean,raw=True)

1  2019-12-01     72.70
   2019-12-05     50.95
   2019-12-12     30.30
   2019-12-18     37.00
   2019-12-26    189.90
2  2019-12-02    186.78
   2019-12-08    158.94
   2019-12-17    145.20
   2019-12-23    172.65
   2019-12-27    383.65
Name: Amount, dtype: float64

What pandarallel does is that is provides you with a new function `parallel_apply` on a dataframe that takes as an input a function that is then executed in parallel by all your CPU Cores by e.g. the group. 

## Conclusion

To sum up we learned in the blog posts some methods to aggregate(group by, rolling aggregations) and transform (merging the data back together) timeseries data to either understand the dataset better or to prepare it for machine learning tasks. We also showed how to parallize some workloads to use all your CPUS on certain operations on your dataset in order to save time. 

I hope that this blog helped you to improve your workflow for time-series data in pandas.